# Note, a geojson is needed for Folium. Json can be found ===> [here:](https://github.com/python-visualization/folium/blob/master/examples/data/world-countries.json) <====

In [1]:
# Needed import
import folium
from folium import plugins
import json
import pandas as pd
import geopandas
import math
import numpy as np
import base64
import matplotlib.pyplot as plt
from folium import IFrame

In [25]:
# Any df with those following columns should be fine, as an example, you can take those
df_final = pd.read_pickle("./data/final_df.pkl")
df_final['profit'] = df_final['export_v'] - df_final['import_v']
df_final['diff_quantity'] = df_final['export_q'] - df_final['import_q']
df_final['import_vpq'] = df_final['import_v'] / df_final['import_q']
df_final['export_vpq'] = df_final['export_v'] / df_final['export_q']
df_final[(df_final['item_trades']=='Wheat')&(df_final['year']==2016)]

,area_code,area_crops,item_crops,year,item_trades,area_harvested,production,yield,export_q,export_v,...,hs12_code,parent_group,child_group,parent_description,child_description,ISO3 Code,profit,diff_quantity,import_vpq,export_vpq
67,1,Armenia,Wheat,2016,Wheat,108049.0,350369.0,32427.0,16.0,6.0,...,100111,10,1001,Cereals,Wheat and meslin,ARM,-60114.0,-288803.0,0.208158,0.375000
1678,2,Afghanistan,Wheat,2016,Wheat,2300210.0,4555110.0,19803.0,0.0,0.0,...,100111,10,1001,Cereals,Wheat and meslin,AFG,-44738.0,-258473.0,0.173086,NaN
3987,3,Albania,Wheat,2016,Wheat,70512.0,275000.0,39000.0,741.0,195.0,...,100111,10,1001,Cereals,Wheat and meslin,ALB,-42843.0,-206267.0,0.207905,0.263158
7031,4,Algeria,Wheat,2016,Wheat,2062179.0,2440097.0,11833.0,NaN,NaN,...,100111,10,1001,Cereals,Wheat and meslin,DZA,NaN,NaN,0.217669,NaN
12789,7,Angola,Wheat,2016,Wheat,4295.0,3333.0,7760.0,NaN,NaN,...,100111,10,1001,Cereals,Wheat and meslin,AGO,NaN,NaN,0.309091,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957759,5801,Least Developed Countries,Wheat,2016,Wheat,5797356.0,13312832.0,22964.0,5433.0,2169.0,...,100111,10,1001,Cereals,Wheat and meslin,NaN,-4102419.0,-18296643.0,0.224269,0.399227
968593,5802,Land Locked Developing Countries,Wheat,2016,Wheat,23428059.0,41924247.0,17895.0,6007991.0,950364.0,...,100111,10,1001,Cereals,Wheat and meslin,NaN,-1257016.0,-3954443.0,0.221570,0.158183
979113,5803,Small Island Developing States,Wheat,2016,Wheat,5.0,9.0,17206.0,2011.0,675.0,...,100111,10,1001,Cereals,Wheat and meslin,NaN,-748579.0,-2791715.0,0.268192,0.335654
987670,5815,Low Income Food Deficit Countries,Wheat,2016,Wheat,48806439.0,142243503.0,29144.0,269639.0,68788.0,...,100111,10,1001,Cereals,Wheat and meslin,NaN,-6783606.0,-31204885.0,0.217712,0.255111


### Below is the main function.
Example of how to use it follows in the next cell.

In [129]:
def show_production_all_countries(df, year, products=None, coloring='profit', filename='data/countries_profit.html'):
    """
    Complete function that takes some predefined df and output a map of the requested food for a given year.
    Show a folium map and then save the plot in an html file
    """
    
    # TODO Import/export VPQ when highlight, Import export meaningful colouring
    
    df = df[df['year'] == year]
    df = df.replace([np.inf, -np.inf], np.nan)
    
    isFirst = True
    
    if products == None:
        products = df.item_trades[:10]
    
    
    #layer_types = ['value', 'quantity']
    m = folium.Map([40,0], zoom_start=2)
    #fg = folium.FeatureGroup(overlay=True, name='groups', control=False)
    #m.add_child(fg)
    
    by_product = df.groupby('item_trades')
    i = 0
    for product in products:
        df_product = by_product.get_group(product)

        countries = geopandas.read_file('data/world-countries.json')
        #cm = countries.merge(df, left_on='id', right_on='iso3_code')
        cm = countries.merge(df_product, left_on='id', right_on='ISO3 Code')
        
        cm['profit'] = cm['profit'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        cm['export_v'] = cm['export_v'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        cm['import_v'] = cm['import_v'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        
        cm['diff_quantity'] = cm['diff_quantity'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        cm['export_q'] = cm['export_q'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        cm['import_q'] = cm['import_q'].apply(lambda value : 0 if math.isnan(value) else f'{int(value):,}')
        
        cm['export_vpq'] = cm['export_vpq'].apply(lambda value : 0 if math.isnan(value) else f'{value:.2f}')
        cm['import_vpq'] = cm['import_vpq'].apply(lambda value : 0 if math.isnan(value) else f'{value:.2f}')
                
        
        cm = cm[['id', 'name', 'geometry', 'item_trades', 'export_q', 'import_q', 'export_v', 'import_v', 'ISO3 Code', 'profit', 'diff_quantity', 'import_vpq', 'export_vpq']]
        cm.to_file("data/countries-production.json", driver='GeoJSON')
        geo_json_data = json.load(open('data/countries-production.json'))

        def my_color_function(feature):
            """Maps high values to green and negative values to red."""
            if len(df[df['ISO3 Code'] == feature['properties']['id']][coloring]) == 0:
                return '#ffffff'
            value = df[(df['ISO3 Code'] == feature['properties']['id'])&(df['item_trades'] == feature['properties']['item_trades'])][coloring].iloc[0]
            feat = feature['properties']['item_trades']
            min_val = (df[df['item_trades']== feat])[coloring].min()
            max_val = (df[df['item_trades']== feat])[coloring].max()
            min_val = 0 if math.isinf(min_val) else min_val
            max_val = 1000 if math.isinf(max_val) else max_val

            if math.isnan(value) or math.isinf(value):
                return "#000000"
            
            if min_val >= 0:
                green = hex( int( (1 - (1 - ((value - min_val) / (max_val - min_val)))**5)* 255) )[2:]
                green = '0' + green if len(green) == 1 else green
                blue = green
                return '#00' + green + blue
            
            if value > 0:
                green = hex(127 + int((value / max_val) * 255 / 2 ))[2:]
                green = '0' + green if len(green) == 1 else green
                return '#00' + green + '00'
            else:
                red = hex(int(127 + (value / min_val) * 255 / 2 ))[2:]
                red = '0' + red if len(red) == 1 else red
                return '#' + red + '00' + '00'

        def my_highlight_function(feature):
            return {
                'fillColor': 'black',
                'color' : 'black',
                'weight' : 3,
                'dashArray' : '5, 5'
                }


        
        layer = folium.GeoJson(
            geo_json_data,
            style_function=lambda feature: {
                'fillColor': my_color_function(feature),
                'color' : 'black',
                'weight' : 1.5,
                'dashArray' : '5, 5'
                },
            overlay=True,
            show=isFirst,
            highlight_function=my_highlight_function,
            tooltip=folium.features.GeoJsonTooltip(
                fields=['name', 'profit', 'export_v', 'import_v', 'diff_quantity', 'export_q', 'import_q', 'export_vpq', 'import_vpq'],
                aliases=['State name:',  'Gross Profit:', 'Export value:', 'Import value:', 'Difference of quantity:', 'Export quantity:', 'Import quantity:', 'Export V/Q:', 'Import V/Q:'],
                )
            )
        #g = plugins.FeatureGroupSubGroup(fg, coloring, show=False)
        #m.add_child(g)
        isFirst = False
        layer.layer_name = product
        layer.add_to(m)
        #layer.add_to(fg)
        
    plugins.Fullscreen(
        position='bottomright',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(m)
       
    folium.LayerControl().add_to(m)
    
    m.save(filename)
    return m

In [130]:
# Example: Show a specific list of product, in year 2017. Color in function of "diff_quantity" column
show_production_all_countries(df_final, 2016, ['Wheat', 'Flour, wheat'], coloring='diff_quantity', filename='wheat_2016_diff_quantity.html')

In [116]:
# Example: Show other features in 1987, color by Gross profit (Profit brut)
show_production_all_countries(df_final, 1987, ['Maté', 'Avocados', 'Pistachios', 'Coffee, green', 'Cereals, breakfast', 'Wheat'], coloring='profit')